In [1]:
import sys

In [2]:
sys.path.append(os.path.join(os.path.abspath(os.path.join('../..')), 'src'))

In [3]:
import numpy
import scipy
import pandas

In [4]:
import mysql_utils
import doc_proc
import init_tdm_tables

In [171]:
from importlib import reload

In [174]:
reload(mysql_utils)

<module 'mysql_utils' from '/home/immersinn/gits/rssfeed_link_collector/src/mysql_utils.py'>

## Init Needed Tools

In [5]:
dd = lambda doc: doc_proc.build_text_feature(doc, 
                                                 components = ['title', 'summary'],
                                                 lower=False, 
                                                 remove_stops=False,
                                                 html_text=True,
                                                 )

In [6]:
dp = init_tdm_tables.DocProcessor()

In [87]:
cnx = mysql_utils.getCnx()
cur = mysql_utils.getCur(cnx)

## Process

In [8]:
def query_wordIDLookup(cur, words):
    format_strings = ','.join(["%s"] * len(words))
    query = ("SELECT id, word FROM words "
             "WHERE word IN ({})".format(format_strings))
    cur.execute(query, list(words))
    cols = cur.column_names
    word_lookup = [{cols[0] : e[0], cols[1] : e[1]} for e in cur.fetchall()]
    word_lookup = {e['id'] : e['word'] for e in word_lookup}
    return(word_lookup)


def query_idWordLookup(cur, wids):
    format_strings = ','.join(["%s"] * len(wids))
    query = ("SELECT id, word FROM words "
             "WHERE id IN ({})".format(format_strings))
    cur.execute(query, list(wids))
    cols = cur.column_names
    word_lookup = [{cols[0] : e[0], cols[1] : e[1]} for e in cur.fetchall()]
    word_lookup = {e['id'] : e['word'] for e in word_lookup}
    return(word_lookup)

In [9]:
stops_lookup = query_wordIDLookup(cur, doc_proc.nltk_stops)

### Get Query Words

In [10]:
doc = {"title" : "FBI refused White House request to knock down recent Trump-Russia stories",
       "summary" : "Washington (CNN) The FBI rejected a recent White House request to publicly knock down media reports about communications between Donald Trump's associates and Russians known to US intelligence during the 2016 presidential campaign, multiple US officials briefed on the matter tell CNN."}

In [11]:
bow = dp.doc2BOW(dd(doc))

In [166]:
orig_query_words = [w for w in \
                    (query_wordIDLookup(cur,
                                       [w for w in bow if w not in doc_proc.nltk_stops])).keys()
                   ]

### Query for Docs that Share Words

In [168]:
def query_docsOnWords(cur, words, word_type="id", exclude_docs=set()):
    
    if word_type == "word":
        word_doc_query = ("SELECT doc_bows.doc_id, doc_bows.wcount "
                          "FROM  doc_bows LEFT JOIN words ON (doc_bows.word_id = words.id) "
                          "WHERE words.word = '{}' ")
    elif word_type == "id":
        word_doc_query = ("SELECT doc_bows.doc_id, doc_bows.wcount "
                          "FROM  doc_bows LEFT JOIN words ON (doc_bows.word_id = words.id) "
                          "WHERE words.id = '{}' ")

    doc_ids = set()
    word_count_store = []
    for word in words:
        cur.execute(word_doc_query.format(word))
        result = mysql_utils.dfDocsFromCursor(cur)
        result = result[[i not in exclude_docs for i in result['doc_id']]]
        if result.shape[0] > 0:
            doc_ids.update(set(result['doc_id']))
            word_count_store.append({'word' : word,
                                     'n_docs' : result.shape[0],
                                     'n_tot' : result['wcount'].sum()})

    doc_ids = [int(i) for i in doc_ids]
    word_count_store = pandas.DataFrame(word_count_store)
        
    return(doc_ids, word_count_store)


def word_summary_info(cur, words, wtype='id', exclude_docs=set(), include_docs=set()):
    """
    CREATE TABLE words_info SELECT doc_bows.word_id, 
    COUNT(doc_bows.doc_id) as n_docs, 
    SUM(doc_bows.wcount) as n_total  
    FROM  doc_bows 
    GROUP BY doc_bows.word_id;
    
    """
    
    if exclude_docs or include_docs:
        if exclude_docs:
            base = "AND doc_bows.doc_id NOT IN ({}) "
            dd = exclude_docs
        elif include_docs:
            base = "AND doc_bows.doc_id IN ({}) "
            dd = include_docs
        dd = [str(int(did)) for did in dd]
        dd = ", ".join(dd)
        ie_where_text = base.format(dd)
    else:
        ie_where_text = ""
        
        
    format_strings = ', '.join(['%s'] * len(words))
    
    if wtype=='word':
        where_clause = "WHERE words.word IN ({}) ".format(format_strings)
        where_clause += ie_where_text
        query = "SELECT doc_bows.word_id, COUNT(DISTINCT doc_bows.doc_id) as n_docs, " +\
                "SUM(doc_bows.wcount) as n_total " +\
                "FROM  doc_bows LEFT JOIN words ON (doc_bows.word_id = words.id) " +\
                where_clause +\
                "GROUP BY doc_bows.word_id"
        
    elif wtype=='id':
        where_clause = "WHERE doc_bows.word_id IN ({}) ".format(format_strings)
        where_clause += ie_where_text
        query = "SELECT doc_bows.word_id, COUNT(DISTINCT doc_bows.doc_id) as n_docs, " +\
                "SUM(doc_bows.wcount) as n_total " +\
                "FROM  doc_bows " +\
                where_clause +\
                "GROUP BY doc_bows.word_id"
            
    cur.execute(query, (words))
    result = mysql_utils.dfDocsFromCursor(cur)
    return(result)

#### Orig Doc

In [169]:
word_count_info = word_summary_info(cur, orig_query_words, wtype='id')

In [170]:
word_count_info.n_docs.describe()

count      31.000000
mean      723.741935
std      1476.303152
min        10.000000
25%        61.500000
50%       233.000000
75%       705.500000
max      7586.000000
Name: n_docs, dtype: float64

In [16]:
# Arbit...this should be "decayed"....see below
ndoc_cutoff = 100

qw_l01 = [int(w) for w in list(word_count_info[word_count_info.n_docs < ndoc_cutoff].word_id)]

In [17]:
# 2nd Pass: Get Doc IDs to use
docs_l01, wcs_l01 = query_docsOnWords(cur, qw_l01, word_type='id')

#### Level 1 Documents

In [19]:
def query_docBOW(cur, doc_id, word_list = []):
    query = "SELECT word_id, wcount FROM doc_bows WHERE doc_id = {}".format(doc_id)
    cur.execute(query)
    cols = cur.column_names
    bow = [{cols[0] : e[0], cols[1] : e[1]} for e in cur.fetchall()]
    if word_list:
        bow = [e for e in bow if e['word_id'] in word_list]
    return(bow)

In [20]:
def query_AllDocWords(cur, doc_ids):
    doc_ids = list(doc_ids)
    format_strings = ','.join(['%s'] * len(doc_ids))
    query = ("SELECT DISTINCT word_id "
             "FROM doc_bows "
             "WHERE doc_id IN ({})".format(format_strings))
    cur.execute(query, (doc_ids))
    words = [e[0] for e in cur.fetchall()]
    return(words)

### This stepo is a bottleneck; can i pre-reduce the query words by using info from the sub-set?

In [41]:
def query_backgroundWordInfo(cur, word_ids):
    wids_format = ", ".join([str(wid) for wid in word_ids])
    query = "SELECT * FROM words_info WHERE word_id IN ({})".format(wids_format)
    cur.execute(query)
    result = mysql_utils.dfDocsFromCursor(cur)
    return(result)

Have the word stats for all words queued up already..in the "words_info" table

In [48]:
query_words = query_AllDocWords(cur, docs_l01)
query_words = [w for w in query_words if w not in stops_lookup.keys()]
len(query_words)

4122

In [176]:
n_docs_tot = mysql_utils.get_total_entries(cur, 'rssfeed_links')
n_docs_tot

38317

In [98]:
bgwi = query_backgroundWordInfo(cur, query_words)
bgwi['frac'] = bgwi.n_docs / n_docs_tot

In [121]:
qwwi = word_summary_info(cur, query_words, wtype='id', include_docs=docs_l01)
qwwi['frac'] = qwwi.n_docs / len(docs_l01)

In [122]:
wi = bgwi.join(qwwi, on=['word_id'], lsuffix='_bg', rsuffix='_qw')

In [123]:
wi.head()

,word_id_bg,n_docs_bg,n_total_bg,frac_bg,word_id_qw,n_docs_qw,n_total_qw,frac_qw
0,1,105,144,0.002740,3.0,80.0,123,0.233918
1,3,7586,11670,0.197980,11.0,6.0,6,0.017544
2,7,91,93,0.002375,15.0,6.0,10,0.017544
3,11,803,932,0.020957,19.0,1.0,1,0.002924
4,12,310,382,0.008090,20.0,44.0,60,0.128655


In [124]:
wi['ratio'] = wi.frac_qw / wi.frac_bg

In [125]:
wi.ratio.describe()

count    1697.000000
mean        6.737447
std        20.801847
min         0.023449
25%         0.732275
50%         1.672209
75%         4.149556
max       336.114035
Name: ratio, dtype: float64

In [146]:
# Restrict query words based on "words_info" table in mysql
# Say, more than 5 docs, but less than 100 (50) (20) docs?
word_filter_l02 = lambda x: (x['n_docs_qw'] >= 3 and x['n_docs_bg'] < 50 and x['ratio'] > 2)

In [154]:
ids2ints = lambda id_series: [int(i) for i in list(id_series)]

In [155]:
qw_l02 = ids2ints(wi.word_id_bg[wi.apply(lambda entry: word_filter_l02(entry), axis=1)])
print(len(qw_l02))

76


In [156]:
docs_l02, wcs_l02 = query_docsOnWords(cur, qw_l02,
                                      word_type="id", exclude_docs=docs_l01)

In [157]:
len(docs_l02)

1732

In [158]:
qww = query_idWordLookup(cur, qw_l02)

## Generate Network from Selected Vocab, Docs

In [164]:
words = set(orig_query_words).copy()
words.update(qw_l01)
words.update(qw_l02)
len(words)

116

In [165]:
for w in words:
    print(w)

3072
1793
number
donald
briefed
1539
1540
communications
3080
fbi
reports
1040
22
535
1302
1561
284
matter
12578
290
refused
rejected
292
2853
301
2863
1585
1586
5427
4148
56
6202
3134
trumps
27973
1095
2632
request
1354
1355
2635
1869
2890
1618
339
2900
2901
2645
russians
1625
trumprussia
house
607
2142
360
2153
1131
officials
multiple
113
1398
us
recent
121
2938
stories
3453
intelligence
campaign
898
known
cnn
902
1159
1670
2702
presidential
400
3220
410
156
publicly
412
668
1693
1694
1193
tell
1196
washington
associates
2481
1716
6327
1207
1725
3262
455
1741
1234
218
987
1754
white
222
990
2268
2272
3034
744
knock
1001
241
2294
2297
media


In [163]:
bows = {}
for d in docs_l01:
    bows[d] = query_docBOW(cur, d, word_list=words)
for d in docs_l02:
    bows[d] = query_docBOW(cur, d, word_list=words)

In [179]:
cnx.close()